In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def cloth_styling(input_image_path, style_image_path, output_image_path):
  %cd /content/drive/MyDrive/Interactive_cloth_segmentation_CV/cloth-segmentation
  import os
  # from tqdm import tqdm
  from tqdm.notebook import tqdm
  from PIL import Image
  import numpy as np

  import torch
  import torch.nn.functional as F
  import torchvision.transforms as transforms

  from data.base_dataset import Normalize_image
  from utils.saving_utils import load_checkpoint_mgpu

  from networks import U2NET
  device = 'cpu'

  image_dir = '/content/drive/MyDrive/Interactive_cloth_segmentation_CV/input/'
  result_dir = '/content/drive/MyDrive/Interactive_cloth_segmentation_CV/output/'
  checkpoint_path = 'cloth_segm_u2net_latest.pth'

  def get_palette(num_cls):
      """ Returns the color map for visualizing the segmentation mask.
      Args:
          num_cls: Number of classes
      Returns:
          The color map
      """
      n = num_cls
      palette = [0] * (n * 3)
      for j in range(0, n):
          lab = j
          palette[j * 3 + 0] = 0
          palette[j * 3 + 1] = 0
          palette[j * 3 + 2] = 0
          i = 0
          while lab:
              palette[j * 3 + 0] |= (((lab >> 0) & 1) << (7 - i))
              palette[j * 3 + 1] |= (((lab >> 1) & 1) << (7 - i))
              palette[j * 3 + 2] |= (((lab >> 2) & 1) << (7 - i))
              i += 1
              lab >>= 3
      return palette


  transforms_list = []
  transforms_list += [transforms.ToTensor()]
  transforms_list += [Normalize_image(0.5, 0.5)]
  transform_rgb = transforms.Compose(transforms_list)

  net = U2NET(in_ch=3, out_ch=4)
  net = load_checkpoint_mgpu(net, checkpoint_path)
  net = net.to(device)
  net = net.eval()

  palette = get_palette(4)

  !rm -rf '/content/drive/MyDrive/Interactive_cloth_segmentation_CV/input/.ipynb_checkpoints'
  images_list = sorted(os.listdir(image_dir))
  print(images_list)
  pbar = tqdm(total=len(images_list))
  def getSegmented_image():
    ret = []
    for image_name in images_list:
        img = Image.open(os.path.join(image_dir, image_name)).convert('RGB')
        img_size = img.size
        img = img.resize((768, 768), Image.BICUBIC)
        image_tensor = transform_rgb(img)
        image_tensor = torch.unsqueeze(image_tensor, 0)
        # A X 1 X B X 1 X C X 1 = A X B X C
        output_tensor = net(image_tensor.to(device))
        # print(output_tensor)
        # output_tensor = F.log_softmax(output_tensor[0], dim=1)
        print(output_tensor[0].shape,"1")
        output_tensor = torch.max(output_tensor[0], dim=1, keepdim=True)[1]
        print(output_tensor.shape,"2")
        output_tensor = torch.squeeze(output_tensor, dim=0)
        print(output_tensor.shape,"3")
        output_tensor = torch.squeeze(output_tensor, dim=0)
        print(output_tensor.shape,"4")
        output_arr = output_tensor.cpu().numpy()

        output_img = Image.fromarray(output_arr.astype('uint8'), mode='L')
        output_img = output_img.resize(img_size, Image.BICUBIC)
        output_img_before = Image.fromarray(output_arr.astype('uint8')*255, mode='L')
        # output_img_before.save(os.path.join(result_dir, image_name[:-4]+'_before.png'))

        output_img.putpalette(palette)
        output_img.save(os.path.join(result_dir, image_name[:-4]+'_segmented.png'))
        ret.append(output_img)
        print(image_name)
        pbar.update(1)
    return ret
  # getSegmented_image()
  pbar.close()

  import os
  import tensorflow as tf
  # Load compressed models from tensorflow_hub
  os.environ['TFHUB_MODEL_LOAD_FORMAT'] = 'COMPRESSED'
  import IPython.display as display

  import matplotlib.pyplot as plt
  import matplotlib as mpl
  mpl.rcParams['figure.figsize'] = (12, 12)
  mpl.rcParams['axes.grid'] = False

  import numpy as np
  import PIL.Image
  import time
  import functools

  def tensor_to_image(tensor):
    tensor = tensor*255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor)>3:
      assert tensor.shape[0] == 1
      tensor = tensor[0]
    return PIL.Image.fromarray(tensor)

  def load_img(path_to_img):
    max_dim = 512
    img = tf.io.read_file(path_to_img)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)

    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    long_dim = max(shape)
    scale = max_dim / long_dim

    new_shape = tf.cast(shape * scale, tf.int32)

    img = tf.image.resize(img, new_shape)
    img = img[tf.newaxis, :]
    return img

  def imshow(image, title=None):
    if len(image.shape or image.size) > 3:
      image = tf.squeeze(image, axis=0)

    plt.imshow(image)
    if title:
      plt.title(title)
  def imshow_pil(image, title=None):
    if len(image.size) > 3:
      image = tf.squeeze(image, axis=0)

    plt.imshow(image)
    if title:
      plt.title(title)
  
  import tensorflow_hub as hub
  hub_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')
  
  content_image = load_img(input_image_path)
  style_image = load_img(style_image_path)
  stylized_image = hub_model(tf.constant(content_image), tf.constant(style_image))[0]
  stylized_image = tensor_to_image(stylized_image)

  %cd /content/drive/MyDrive/Interactive_cloth_segmentation_CV/U-2-Net
  # U-2-Net -> Saliency
  # U-2-Net only segmentation -> Segmentation

  import os
  from skimage import io, transform
  import torch
  import torchvision
  from torch.autograd import Variable
  import torch.nn as nn
  import torch.nn.functional as F
  from torch.utils.data import Dataset, DataLoader
  from torchvision import transforms#, utils
  # import torch.optim as optim

  import numpy as np
  from PIL import Image
  import glob

  from data_loader import RescaleT
  from data_loader import ToTensor
  from data_loader import ToTensorLab
  from data_loader import SalObjDataset

  from model import U2NET # full size version 173.6 MB
  from model import U2NETP # small version u2net 4.7 MB

  # normalize the predicted SOD probability map
  def normPRED(d):
      ma = torch.max(d)
      mi = torch.min(d)

      dn = (d-mi)/(ma-mi)

      return dn

  def save_output(image_name,pred,d_dir):

      predict = pred
      predict = predict.squeeze()
      predict_np = predict.cpu().data.numpy()

      im = Image.fromarray(predict_np*255).convert('RGB')
      img_name = image_name.split(os.sep)[-1]
      image = io.imread(image_name)
      imo = im.resize((image.shape[1],image.shape[0]),resample=Image.BILINEAR)

      pb_np = np.array(imo)

      aaa = img_name.split(".")
      bbb = aaa[0:-1]
      imidx = bbb[0]
      for i in range(1,len(bbb)):
          imidx = imidx + "." + bbb[i]

      imo.save(d_dir+imidx+'_saliency.png')
      return imo

  def get_saliency_maps_name(image_name):

      # --------- 1. get image path and name ---------
      model_name='u2net'#u2netp



      image_dir =  '../input'
      prediction_dir =  '../output/'
      model_dir = os.path.join(os.getcwd(), 'saved_models', model_name, model_name + '.pth')
      print(os.getcwd())
      img_name_list = glob.glob(image_dir + os.sep + '*')
      print(img_name_list)

      # --------- 2. dataloader ---------
      #1. dataloader
      test_salobj_dataset = SalObjDataset(img_name_list = img_name_list,
                                          lbl_name_list = [],
                                          transform=transforms.Compose([RescaleT(320),
                                                                        ToTensorLab(flag=0)])
                                          )
      test_salobj_dataloader = DataLoader(test_salobj_dataset,
                                          batch_size=1,
                                          shuffle=False,
                                          num_workers=1)

      # --------- 3. model define ---------
      if(model_name=='u2net'):
          print("...load U2NET---173.6 MB")
          net = U2NET(3,1)
      elif(model_name=='u2netp'):
          print("...load U2NEP---4.7 MB")
          net = U2NETP(3,1)

      if torch.cuda.is_available():
          net.load_state_dict(torch.load(model_dir))
          net.cuda()
      else:
          net.load_state_dict(torch.load(model_dir, map_location='cpu'))
      net.eval()

      # --------- 4. inference for each image ---------
      ret = []
      for i_test, data_test in enumerate(test_salobj_dataloader):
          if(img_name_list[i_test].split(os.sep)[-1]==image_name):

            print("inferencing:",img_name_list[i_test].split(os.sep)[-1])

            inputs_test = data_test['image']
            inputs_test = inputs_test.type(torch.FloatTensor)

            if torch.cuda.is_available():
                inputs_test = Variable(inputs_test.cuda())
            else:
                inputs_test = Variable(inputs_test)

            d1,d2,d3,d4,d5,d6,d7= net(inputs_test)

            # normalization
            pred = d1[:,0,:,:]
            pred = normPRED(pred)

            # save results to test_results folder
            if not os.path.exists(prediction_dir):
                os.makedirs(prediction_dir, exist_ok=True)
            img = save_output(img_name_list[i_test],pred,prediction_dir)
            ret.append(img)
            del d1,d2,d3,d4,d5,d6,d7
      return ret
  def get_saliency_maps():

      # --------- 1. get image path and name ---------
      model_name='u2net'#u2netp



      image_dir =  '../input'
      prediction_dir =  '../output/'
      model_dir = os.path.join(os.getcwd(), 'saved_models', model_name, model_name + '.pth')
      print(os.getcwd())
      img_name_list = glob.glob(image_dir + os.sep + '*')
      print(img_name_list)

      # --------- 2. dataloader ---------
      #1. dataloader
      test_salobj_dataset = SalObjDataset(img_name_list = img_name_list,
                                          lbl_name_list = [],
                                          transform=transforms.Compose([RescaleT(320),
                                                                        ToTensorLab(flag=0)])
                                          )
      test_salobj_dataloader = DataLoader(test_salobj_dataset,
                                          batch_size=1,
                                          shuffle=False,
                                          num_workers=1)

      # --------- 3. model define ---------
      if(model_name=='u2net'):
          print("...load U2NET---173.6 MB")
          net = U2NET(3,1)
      elif(model_name=='u2netp'):
          print("...load U2NEP---4.7 MB")
          net = U2NETP(3,1)

      if torch.cuda.is_available():
          net.load_state_dict(torch.load(model_dir))
          net.cuda()
      else:
          net.load_state_dict(torch.load(model_dir, map_location='cpu'))
      net.eval()

      # --------- 4. inference for each image ---------
      ret = []
      for i_test, data_test in enumerate(test_salobj_dataloader):
          

          print("inferencing:",img_name_list[i_test].split(os.sep)[-1])

          inputs_test = data_test['image']
          inputs_test = inputs_test.type(torch.FloatTensor)

          if torch.cuda.is_available():
              inputs_test = Variable(inputs_test.cuda())
          else:
              inputs_test = Variable(inputs_test)

          d1,d2,d3,d4,d5,d6,d7= net(inputs_test)

          # normalization
          pred = d1[:,0,:,:]
          pred = normPRED(pred)

          # save results to test_results folder
          if not os.path.exists(prediction_dir):
              os.makedirs(prediction_dir, exist_ok=True)
          img = save_output(img_name_list[i_test],pred,prediction_dir)
          ret.append(img)
          del d1,d2,d3,d4,d5,d6,d7
      return ret

  %cd /content/drive/MyDrive/Interactive_cloth_segmentation_CV/cloth-segmentation
  segmented_images = getSegmented_image()
  len(segmented_images)
  %cd /content/drive/MyDrive/Interactive_cloth_segmentation_CV/U-2-Net
  saliency_maps = get_saliency_maps()

  # print(saliency_maps[0].size, stylized_image.size, segmented_images[0].size)
  #@title ## Input Image Paths
  stylized_images = []

  style_image = load_img(style_image_path)

  content_Image_path = "/content/drive/MyDrive/Interactive_cloth_segmentation_CV/input/"
  reuse_old_image = False
  for i in range(len(saliency_maps)):
    content = load_img(content_Image_path+images_list[i])
    stylized = hub_model(tf.constant(content), tf.constant(style_image))[0]
    stylized = tensor_to_image(stylized)
    stylized_images.append(stylized)
    saliency_maps[i] = saliency_maps[i].resize((stylized.size[0],stylized.size[1]),Image.BICUBIC)
    segmented_images[i] = segmented_images[i].resize((stylized.size[0],stylized.size[1]),Image.BICUBIC)
  if(not(reuse_old_image)):
    inputs_img = []
    inputs_img2 = []
    for i in range(len(images_list)):
      img = Image.open(os.path.join(image_dir, images_list[i])).convert('RGB')
      img2 = Image.open(os.path.join(image_dir, images_list[i])).convert('RGB')
      img = img.resize((stylized_images[i].size[0],stylized_images[i].size[1]),Image.BICUBIC)
      inputs_img.append(img)
      inputs_img2.append(img2)
  for i in range(len(segmented_images)):
    segmented_images[i] = segmented_images[i].convert('RGB')

  
  plt.subplot(1, 3, 1)
  imshow_pil(saliency_maps[0],"saliency maps")
  plt.subplot(1, 3, 2)
  imshow_pil(stylized_images[0],"stylized image")
  plt.subplot(1, 3, 3)
  imshow_pil(segmented_images[0],"segmented image")

  # plt.subplot(1, 3, 1)
  from matplotlib.patches import Rectangle
  from matplotlib.gridspec import GridSpec
  na = np.array(segmented_images[0].convert('RGB'))
  colours, counts = np.unique(na.reshape(-1,3), axis=0, return_counts=1)
  # print(colours)
  gs = GridSpec(6,1)

  fig = plt.figure(figsize = (10,10))
  ax1 = fig.add_subplot(gs[:-1,:]) ##for the plot
  ax2 = fig.add_subplot(gs[-1,:]) 
  legend_data = []
  # print(colours)
  ax1.imshow(segmented_images[0])
  for i in range(len(colours)):
    if(colours[i][0]==0 and colours[i][1]==0 and colours[i][2]==0):
      continue
    temp = [0,colours[i],i]
    legend_data.append(temp)

  handles = [
      Rectangle((0,0),1,1, color = tuple((v/255 for v in c))) for k,c,n in legend_data
  ]
  labels = [n for k,c,n in legend_data]
  ax2.legend(handles,labels, mode='expand', ncol=3)
  ax2.axis('off')

  plt.legend(handles,labels)
  plt.show()

  ctr =0
  chosen_segment =  2
  segmented_styled = []
  for i in range(len(segmented_images)):
    newImage = segmented_images[i].copy()
    pix_seg = segmented_images[i].load()
    pix_sty = stylized_images[i].load()
    
    pix_seg_sty = newImage.load()
    for x in range((segmented_images[i].size[0])):
      for y in range((segmented_images[i].size[1])):
        if(pix_seg[x,y]==(colours[chosen_segment][0],colours[chosen_segment][1],colours[chosen_segment][2])):
          # print(pix_sty[x,y])
          # print((pix_seg[x,y]))
          pix_seg_sty[x,y] = pix_sty[x,y]

          # break
    segmented_styled.append(newImage)

  fig = plt.figure(figsize = (10,10))
  ax1 = fig.add_subplot(gs[:-1,:]) ##for the plot
  ax2 = fig.add_subplot(gs[-1,:]) 
  legend_data = []
  # print(colours)
  ax1.imshow(segmented_styled[0])
  for i in range(len(colours)):
    if(colours[i][0]==0 and colours[i][1]==0 and colours[i][2]==0):
      continue
    temp = [0,colours[i],i]
    legend_data.append(temp)

  handles = [
      Rectangle((0,0),1,1, color = tuple((v/255 for v in c))) for k,c,n in legend_data
  ]
  labels = [n for k,c,n in legend_data]
  ax2.legend(handles,labels, mode='expand', ncol=3)
  ax2.axis('off')

  plt.legend(handles,labels)
  plt.show()


  import colorsys

  res = []
  import copy
  for i in range(len(segmented_images)):
    pix_unstyled = segmented_images[i].load()
    pix_seg = segmented_styled[i].load()
    pix_saliency = saliency_maps[i].convert(mode='L').load()
    pix_sty = stylized_images[i].load()
    pix_inp = inputs_img[i].load()
    newImage = copy.deepcopy(inputs_img[i])
    lum_image = copy.deepcopy(inputs_img[i])
    pix_lum = lum_image.convert(mode="L").load()
    # environment = brightness(inputs_img[i])
    pix_nimg = newImage.load()
    for x in range((segmented_images[i].size[0])):
      for y in range((segmented_images[i].size[1])):
        if(pix_unstyled[x,y]==(colours[chosen_segment][0],colours[chosen_segment][1],colours[chosen_segment][2])):
          temp = colorsys.rgb_to_hsv(pix_seg[x,y][0]/255, pix_seg[x,y][1]/255, pix_seg[x,y][2]/255)
          temp2 = colorsys.rgb_to_hsv(pix_inp[x,y][0]/255, pix_inp[x,y][1]/255, pix_inp[x,y][2]/255)
          # print(temp2[2])
          final = colorsys.hsv_to_rgb(temp[0], temp[1], temp2[2])
          # print(pix_saliency[x,y])
          pix_nimg[x,y] = tuple(int(i) for i in pix_seg[x,y])
          if(pix_saliency[x,y]==255):
            pix_nimg[x,y] = tuple(int(i) for i in pix_seg[x,y])
          else:
            saliency_binary_val = (pix_saliency[x,y]/255)
            final = tuple((saliency_binary_val*pix_seg[x,y][i] + (1-saliency_binary_val)*pix_inp[x,y][i]) for i in range(3))
            # print(final)
          # print(pix_seg[x,y], final)                           
            pix_inp[x,y] = tuple(int(i) for i in final)
          
          # print(pix_nimg[x,y])
  # newImage = newImage.convert('RGB')
  res.append(newImage)
  newImage.save(output_image_path)
  return newImage



In [ ]:
def pose_transfer(input_image_path,person):
  from PIL import Image
  %cd /content
  !git clone https://github.com/PeikeLi/Self-Correction-Human-Parsing
  %cd Self-Correction-Human-Parsing
  !mkdir checkpoints
  !mkdir inputs
  !mkdir outputs
  dataset = 'lip'         #select from ['lip', 'atr', 'pascal']
  !gdown --id 1h0QEmZQleNCWBY-Fvv7yrRyRT_kDZHez -O checkpoints/final.pth
  !pip install ninja
  !pip install tensorboardX
  %cd /content/Self-Correction-Human-Parsing
  !python3 simple_extractor.py --dataset 'lip' --model-restore 'checkpoints/final.pth' --input-dir 'inputs' --output-dir 'outputs'
  %cd ..
  from pathlib import Path
  import os

  if(person == "man"):
    my_file1 = Path("/content/drive/MyDrive/dressing-in-order/DATA_ROOT/test/fashionMENShortsid0000098901_1front.jpg")
    if my_file1.is_file():
      os.remove("/content/drive/MyDrive/dressing-in-order/DATA_ROOT/test/fashionMENShortsid0000098901_1front.jpg")
      print("removed from test")

    uploaded1 = Image.open(input_image_path) 
    uploaded1 = uploaded1.save("/content/drive/MyDrive/dressing-in-order/DATA_ROOT/test/fashionMENShortsid0000098901_1front.jpg")

  elif(person == "women"):
    my_file1 = Path("/content/drive/MyDrive/dressing-in-order/DATA_ROOT/test/fashionWOMENTees_Tanksid0000336104_1front.jpg")
    if my_file1.is_file():
      os.remove("/content/drive/MyDrive/dressing-in-order/DATA_ROOT/test/fashionWOMENTees_Tanksid0000336104_1front.jpg")
      print("removed from test")

    uploaded1 = Image.open(input_image_path) 
    uploaded1 = uploaded1.save("/content/drive/MyDrive/dressing-in-order/DATA_ROOT/test/fashionWOMENTees_Tanksid0000336104_1front.jpg")


  !gdown --id 1SkCnqhGIHOSnlRQL3GkjYfj899cOWYTF -O checkpoints.zip
  !gdown --id 1pRIwjWAos9q5or04fnK44SPukSI616bS -O datasets.zip
  !gdown --id 1bTFAC6cqdV57FbbiUfSPtnLJG0AI-jFK -O Images.zip
  !gdown --id 1p2gAcET7JTuHRCsXMDRf4-Pwftk1_rQW -O models.zip
  !gdown --id 1FeLBMHp2l4CZrV0Nvr8eIXl4Wm6dKVBm -O options.zip
  !gdown --id 1PAY_1DQ6LiY6_fxbz8m4U7dxEIftd1i1 -O scripts.zip
  !gdown --id 1H6baRQN5naV22e5e5fgeD0moObAZ4BR3 -O tools.zip
  !gdown --id 1wm_uKVE1S4a7nlwZfGK6pbTyRPSBD6AZ -O utils.zip
  !gdown --id 1t6SN1hJvfp-zzDsn8v7LE5snImu5i1kw -O pretrained_models.zip

  ! unzip checkpoints.zip 
  ! unzip datasets.zip 
  ! unzip Images.zip 
  ! unzip models.zip 
  ! unzip options.zip 
  ! unzip scripts.zip 
  ! unzip tools.zip 
  ! unzip utils.zip -d /content/utils
  ! unzip pretrained_models.zip -d /content/pretrained_models

  import sys
  sys.path.append('../')
  import torch
  from models.dior_model import DIORModel
  import os, json
  import torch.nn.functional as F
  from PIL import Image
  import matplotlib.pyplot as plt
  %matplotlib inline


  dataroot = '/content/drive/MyDrive/dressing-in-order/DATA_ROOT'
  exp_name = 'DIOR_64' # DIORv1_64
  epoch = 'latest'
  netG = 'dior' # diorv1
  ngf = 64

  ## this is a dummy "argparse" 
  class Opt:
      def __init__(self):
          pass
  if True:
      opt = Opt()
      opt.dataroot = dataroot
      opt.isTrain = False
      opt.phase = 'test'
      opt.n_human_parts = 8; opt.n_kpts = 18; opt.style_nc = 64
      opt.n_style_blocks = 4; opt.netG = netG; opt.netE = 'adgan'
      opt.ngf = ngf
      opt.norm_type = 'instance'; opt.relu_type = 'leakyrelu'
      opt.init_type = 'orthogonal'; opt.init_gain = 0.02; opt.gpu_ids = [0]
      opt.frozen_flownet = True; opt.random_rate = 1; opt.perturb = False; opt.warmup=False
      opt.name = exp_name
      opt.vgg_path = ''; opt.flownet_path = 'pretrained_models/flownet.pt'
      opt.checkpoints_dir = 'checkpoints'
      opt.frozen_enc = True
      opt.load_iter = 0
      opt.epoch = epoch
      opt.verbose = False

  # create model
  model = DIORModel(opt)
  model.setup(opt)

  # load data
  from datasets.deepfashion_datasets import DFVisualDataset
  Dataset = DFVisualDataset
  ds = Dataset(dataroot=dataroot, dim=(256,176), n_human_part=8)

  # preload a set of pre-selected models defined in "standard_test_anns.txt" for quick visualizations 
  inputs = dict()
  for attr in ds.attr_keys:
      inputs[attr] = ds.get_attr_visual_input(attr)
      
  # define some tool functions for I/O
  def load_img(pid, ds):
      if isinstance(pid,str): # load pose from scratch
          return None, None, load_pose_from_json(pid)
      if len(pid[0]) < 10: # load pre-selected models
          person = inputs[pid[0]]
          person = (i.cuda() for i in person)
          pimg, parse, to_pose = person
          pimg, parse, to_pose = pimg[pid[1]], parse[pid[1]], to_pose[pid[1]]
      else: # load model from scratch
          person = ds.get_inputs_by_key(pid[0])
          person = (i.cuda() for i in person)
          pimg, parse, to_pose = person
      return pimg.squeeze(), parse.squeeze(), to_pose.squeeze()

  def load_pose_from_json(ani_pose_dir):
      with open(ani_pose_dir, 'r') as f:
          anno = json.load(f)
      len(anno['people'][0]['pose_keypoints_2d'])
      anno = list(anno['people'][0]['pose_keypoints_2d'])
      x = np.array(anno[1::3])
      y = np.array(anno[::3])

      coord = np.concatenate([x[:,None], y[:,None]], -1)
      #import pdb; pdb.set_trace()
      #coord = (coord * 1.1) - np.array([10,30])[None, :]
      pose  = pose_utils.cords_to_map(coord, (256,176), (256, 256))
      pose = np.transpose(pose,(2, 0, 1))
      pose = torch.Tensor(pose)
      return pose

  def plot_img(pimg=[], gimgs=[], oimgs=[], gen_img=[], pose=None,folder='',number=''):
      if pose != None:
          import utils.pose_utils as pose_utils
          print(pose.size())
          kpt = pose_utils.draw_pose_from_map(pose.cpu().numpy().transpose(1,2,0),radius=6)
          kpt = kpt[0]
      if not isinstance(pimg, list):
          pimg = [pimg]
      if not isinstance(gen_img, list):
          gen_img = [gen_img]
      out = pimg + gimgs + oimgs + gen_img
      if out:
          out = torch.cat(out, 2).float().cpu().detach().numpy()
          out = (out + 1) / 2 # denormalize
          out = np.transpose(out, [1,2,0])

          if pose != None:
              out = np.concatenate((kpt, out),1)
      else:
          out = kpt
      fig = plt.figure(figsize=(6,4), dpi= 100, facecolor='w', edgecolor='k')
      plt.axis('off')
      plt.imshow(out)
      result_dir="Results/"+folder+"/"+"Pose"+number
      fig.savefig(result_dir)

  # define dressing-in-order function (the pipeline)
  def dress_in_order(model, pid, pose_id=None, gids=[], ogids=[], order=[5,1,3,2], perturb=False):
      PID = [0,4,6,7]
      GID = [2,5,1,3]
      # encode person
      pimg, parse, from_pose = load_img(pid, ds)
      if perturb:
          pimg = perturb_images(pimg[None])[0]
      if not pose_id:
          to_pose = from_pose
      else:
          to_img, _, to_pose = load_img(pose_id, ds)
      psegs = model.encode_attr(pimg[None], parse[None], from_pose[None], to_pose[None], PID)

      # encode base garments
      gsegs = model.encode_attr(pimg[None], parse[None], from_pose[None], to_pose[None])
    
      
      # swap base garment if any
      gimgs = []
      for gid in gids:
          _,_,k = gid
          gimg, gparse, pose =  load_img(gid, ds)
          seg = model.encode_single_attr(gimg[None], gparse[None], pose[None], to_pose[None], i=gid[2])
          gsegs[gid[2]] = seg
          gimgs += [gimg * (gparse == gid[2])]

      # encode garment (overlay)
      garments = []
      over_gsegs = []
      oimgs = []
      for gid in ogids:
          oimg, oparse, pose = load_img(gid, ds)
          oimgs += [oimg * (oparse == gid[2])]
          seg = model.encode_single_attr(oimg[None], oparse[None], pose[None], to_pose[None], i=gid[2])
          over_gsegs += [seg]
      
      gsegs = [gsegs[i] for i in order] + over_gsegs
      gen_img = model.netG(to_pose[None], psegs, gsegs)
      
      return pimg, gimgs, oimgs, gen_img[0], to_pose

  !mkdir Results
  !mkdir Results/Pose

  import numpy as np
  %matplotlib notebook
  %matplotlib inline
  # person id
  if person == "men":
    pid = ('plain', 1, None)
  elif person == "women":
    pid = ('plain', 3, None)
  # pid = ("lace", 0, None) 
  # pid= ("lace" ,1 , None)
  # pid= ("print" ,0 , None)
  for i in range(0,4):
    pose_id = ("lace", i, None) 
    pimg, gimgs, oimgs, gen_img, pose = dress_in_order(model, pid, pose_id=pose_id)
    plot_img(pimg, gimgs, oimgs, gen_img, pose,"Pose",str(i+1))

  for i in range(0,4):
    pose_id = ("plaid", i, None) 
    pimg, gimgs, oimgs, gen_img, pose = dress_in_order(model, pid, pose_id=pose_id)
    plot_img(pimg, gimgs, oimgs, gen_img, pose,"Pose",str(i+5))

  # pid= ("plaid" ,0, None)

  for i in range(0,4):
    pose_id = ("plain", i, None) 
    pimg, gimgs, oimgs, gen_img, pose = dress_in_order(model, pid, pose_id=pose_id)
    plot_img(pimg, gimgs, oimgs, gen_img, pose,"Pose",str(i+9))

  for i in range(0,4):
    pose_id = ("pattern", i, None) 
    pimg, gimgs, oimgs, gen_img, pose = dress_in_order(model, pid, pose_id=pose_id)
    plot_img(pimg, gimgs, oimgs, gen_img, pose,"Pose",str(i+13))

  for i in range(0,4):
    pose_id = ("strip", i, None) 
    pimg, gimgs, oimgs, gen_img, pose = dress_in_order(model, pid, pose_id=pose_id)
    plot_img(pimg, gimgs, oimgs, gen_img, pose,"Pose",str(i+17))

  # pid= ("gfla" ,1 , None)

  for i in range(0,4):
    pose_id = ("print", i, None) 
    pimg, gimgs, oimgs, gen_img, pose = dress_in_order(model, pid, pose_id=pose_id)
    plot_img(pimg, gimgs, oimgs, gen_img, pose,"Pose",str(i+21))

  # pid= ("plain" ,2 , None)

  for i in range(0,4):
    pose_id = ("collar", i, None) 
    pimg, gimgs, oimgs, gen_img, pose = dress_in_order(model, pid, pose_id=pose_id)
    plot_img(pimg, gimgs, oimgs, gen_img, pose,"Pose",str(i+25))


  for i in range(0,4):
    pose_id = ("gfla", i, None) 
    pimg, gimgs, oimgs, gen_img, pose = dress_in_order(model, pid, pose_id=pose_id)
    plot_img(pimg, gimgs, oimgs, gen_img, pose,"Pose",str(i+29))

  # pid= ("pattern" ,1 , None)

  for i in range(0,4):
    pose_id = ("flower", i, None) 
    pimg, gimgs, oimgs, gen_img, pose = dress_in_order(model, pid, pose_id=pose_id)
    plot_img(pimg, gimgs, oimgs, gen_img, pose,"Pose",str(i+33))

  for i in range(0,4):
    pose_id = ("jacket", i, None) 
    pimg, gimgs, oimgs, gen_img, pose = dress_in_order(model, pid, pose_id=pose_id)
    plot_img(pimg, gimgs, oimgs, gen_img, pose,"Pose",str(i+37))


  # !zip -r /content/results.zip /content/Results
  # from google.colab import files
  # files.download("/content/results.zip")
  # files.download(input_image_path)



In [ ]:
def run_final(input_image_path, style_image_path, output_image_path,person):
  cloth_styling(input_image_path, style_image_path, output_image_path)
  pose_transfer(output_image_path,person)

style_image = "/content/drive/MyDrive/Interactive_cloth_segmentation_CV/style/style_image.jpg"

man_input_image = "/content/drive/MyDrive/Interactive_cloth_segmentation_CV/input/fashionMENShortsid0000098901_1front.jpg"
man_output_image = "/content/fashionMENShortsid0000098901_1front.jpg"

women_input_image = "/content/drive/MyDrive/Interactive_cloth_segmentation_CV/input/fashionWOMENTees_Tanksid0000336104_1front.jpg"
women_output_image = "/content/Output_Stylized_Image_women.jpg"

In [ ]:
!pip install flask-ngrok

In [ ]:
!pip install pyngrok==4.1.1
!ngrok authtoken '2Ooqyfp3chBOMXWI4u58Pi7rCmk_4MJ6ho5vLENbTD9SnZrGU'

In [ ]:
from flask import Flask, request, render_template, session, send_file, jsonify
import os
from werkzeug.utils import secure_filename

from pyngrok import ngrok
from flask_ngrok import run_with_ngrok

# # Define allowed files
ALLOWED_EXTENSIONS = {'txt', 'pdf', 'png', 'jpg', 'jpeg', 'gif'}


app = Flask(__name__, static_folder="/content/drive/MyDrive/ir_project/static",template_folder="/content/drive/MyDrive/ir_project/templates")
app.config['UPLOAD_FOLDER'] = "/content/drive/MyDrive/Interactive_cloth_segmentation_CV/input/"
app.secret_key = 'This is your secret key to utilize session in Flask'
run_with_ngrok(app)  


@app.route('/')
def home():
    return render_template("home.html")


@app.route("/submit", methods=["POST"])
def output():
    img1 = request.files["image1"]
    img_filename = secure_filename(img1.filename)
    img1.save(os.path.join(app.config['UPLOAD_FOLDER'], img_filename))
    session['img1_file_path'] = os.path.join(app.config['UPLOAD_FOLDER'], img_filename)
    img2 = request.files["image2"]
    img2_filename = secure_filename(img2.filename)
    img2.save(os.path.join(app.config['UPLOAD_FOLDER'], img2_filename))
    session['img2_file_path'] = os.path.join(app.config['UPLOAD_FOLDER'], img2_filename)
    run_final(session['img1_file_path'], session['img2_file_path'], women_output_image, request.form('t_model'))
    return render_template("output.html", img1="Pose1.png",img2="Pose2.png", img3="Pose3.png", img4="Pose4.png")

@app.route('/image/<filename>')
def serve_image(filename):
    path = "/content/Results/Pose/"+filename
    print(path)
    return send_file(path)

@app.route("/getData", methods=["GET"])
def data_to_colab():
    images = [{'url': session["img1_file_path"]}, {'url': session["img2_file_path"]}]
    return jsonify(images=images)


app.run()
